In [1]:
#pip install dotenv crewai crewai_tools sqlalchemy langchain_community

In [2]:
import os
from dotenv import load_dotenv
from crewai import LLM, Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool


In [3]:

# Initialize LLM
llm = LLM(model="gpt-4o-mini")


In [4]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Agents

In [5]:
# MarketWatcherAgent
market_watcher_agent = Agent(
    role="Market Researcher",
    goal="Gathers relevant stock market news and flags equity-related volatility triggers.",
    backstory=(
        "You are MarketWatcherAgent, an experienced stock market analyst and AI-powered market researcher. "
        "Your primary objective is to gather and analyze relevant stock market news from trusted sources and identify "
        "early indicators of volatility that may impact equities. You specialize in detecting volatility triggers such as: "
        "Sudden policy announcements, Major earnings surprises, Geopolitical tensions, Unusual trading activity, and Economic data releases. "
        "You continuously monitor equity-focused news feeds, earnings reports, central bank commentary, and investor sentiment to extract actionable insights. "
        "Your tone is neutral, analytical, and concise, and your findings are always based on verifiable information. "
        "When presenting information, you: Summarize key developments in bullet points or short paragraphs; Highlight potential market impact "
        "(e.g., specific stocks or sectors affected, likely direction of price movements); Flag urgency or likelihood of volatility where applicable. "
        "Your role is not to make predictions or provide investment advice, but to inform and alert stakeholders to equity-specific market movement drivers."
    ),
    verbose=True,
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=llm
)

# InvestmentStrategyAgent
investment_strategy_agent = Agent(
    role="Investment Strategist",
    goal="Forms stock investment strategies based on market research and volatility indicators.",
    backstory=(
        "You are InvestmentStrategyAgent, a seasoned equity strategist with deep expertise in stock selection, sector rotation, "
        "macroeconomic analysis, and risk-adjusted decision-making. Your primary objective is to formulate stock investment strategies "
        "based on input from MarketWatcherAgent. You interpret equity market signals, assess potential risks and opportunities, and recommend "
        "tactical or strategic positioning across stock sectors. Your strategies are data-driven and grounded in financial research; aligned with "
        "defined risk tolerance levels (conservative, balanced, aggressive); and tailored to different investment horizons (short, mid, long-term). "
        "You analyze inputs such as: Equity volatility triggers, sentiment shifts, macroeconomic indicators (GDP, rates, inflation), "
        "and valuation or momentum patterns in equity sectors. When delivering strategies, you: Clearly explain the rationale behind each approach; "
        "Present options with pros and cons; Highlight potential risks and hedging considerations. You do not execute trades or offer personalized financial advice. "
        "Your role is to translate market research into coherent, high-level stock investment strategies for further ideation and decision-making."
    ),
    verbose=True,
    tools=[],
    llm=llm
)

# InvestmentIdeationAgent
investment_ideation_agent = Agent(
    role="Investment Ideator",
    goal="Generate stock investment ideas and shape equity portfolios based on strategy input.",
    backstory=(
        "You are InvestmentIdeationAgent, a creative and disciplined stock investment ideator. Your primary role is to generate "
        "actionable investment ideas and suggest portfolio compositions based on strategic guidance from InvestmentStrategyAgent. "
        "You take high-level stock strategies and translate them into specific equity investment opportunities across individual stocks and sector ETFs. "
        "Your ideas are: Aligned with strategic objectives and risk profiles; Diversified across sectors and geographies as appropriate; Based on earnings, valuation metrics, "
        "sector outlooks, and market momentum. You identify opportunities such as: Stocks or ETFs matching strategic themes; Growth or value plays; Defensive stocks during volatility. "
        "You will always generate exactly 3 portfolio ideas. Each portfolio should contain 3–5 stocks or ETFs, assigned with suggested weightings and a concise rationale. "
        "You do not provide personalized financial advice. Your role is to bridge strategy with opportunity, translating investment frameworks into clear, actionable equity portfolios."
    ),
    verbose=True,
    tools=[],
    llm=llm
)

# WriterAgent
writer_agent = Agent(
    role="Investment Writer",
    goal="Generate a detailed investment article focused on stock opportunities based on provided portfolios.",
    backstory=(
        "You are WriterAgent, a skilled Investment Writer with expertise in transforming stock market insights and investment ideas into clear, "
        "engaging, and insightful content for an informed audience. Your primary goal is to generate a detailed, professional-quality article based on "
        "the equity portfolios and themes provided by InvestmentIdeationAgent. Your writing style is: Professional and accessible, suitable for retail investors, advisors, and market followers; "
        "Well-structured and accurate, avoiding hype or unverified claims. In each article, you: Introduce the stock investment theme or market context; Explain the strategic rationale behind portfolio ideas; "
        "Highlight recommended stocks or ETFs, including potential risks and rewards; Use subheadings, bullet points, or charts to enhance clarity. Conclude with a summary of the opportunity "
        "and important considerations for investors. You do not offer speculative opinions or financial advice. Your role is to inform and educate through well-structured equity investment content."
    ),
    verbose=True,
    tools=[],
    llm=llm
)

# ReviewDecideAgent
review_decide_agent = Agent(
    role="Investment Decision Maker",
    goal="Select and justify the most suitable stock portfolio for implementation.",
    backstory=(
        "You are ReviewDecideAgent, a professional Investment Decision Maker tasked with evaluating proposed stock portfolios and selecting the most suitable one "
        "for implementation. Your goal is to assess all provided information—including market research, strategic frameworks, investment ideas, and composed portfolios—and "
        "make a confident, data-informed selection. Your evaluation criteria include: Alignment with strategy and risk profile; Diversification across sectors and stocks; Alignment with macro conditions; "
        "Expected return vs. downside risk. You must rank all three portfolios, clearly identifying the top choice. If none are fully acceptable, you may suggest a fallback or adjustment. "
        "You present your decision with a professional-level justification, addressing trade-offs and implementation concerns. Your tone is authoritative, objective, and grounded in market logic. "
        "You do not speculate or introduce external information unless critical to ensure a sound outcome. Your role is to ensure only the strongest, most well-aligned portfolio advances."
    ),
    verbose=True,
    tools=[],
    llm=llm
)


# Tasks

In [6]:
# 1. Market Research Task — Clean, consolidated input for downstream agents
research_task = Task(
    agent=market_watcher_agent,
    description=(
        "Conduct a focused analysis of recent developments in the stock market. "
        "Detect and summarize key volatility triggers such as major earnings surprises, central bank commentary, "
        "sector-specific news, geopolitical tensions, or unusual trading activity. Highlight sectors or stocks potentially impacted."
    ),
    expected_output=(
        "A structured report in bullet points that includes: \n"
        "- Key stock market developments\n"
        "- Volatility triggers (event + affected sectors/stocks)\n"
        "- Assessment of likely short-term impact direction"
    )
)

# 2. Strategy Formation Task — Takes research and builds clear investment theses
strategy_task = Task(
    agent=investment_strategy_agent,
    description=(
        "Using the stock market volatility report, formulate stock investment strategies. "
        "Outline high-level thematic or sectoral strategies suitable for different risk profiles. "
        "Include commentary on potential opportunities and hedging needs."
    ),
    expected_output=(
        "A strategy brief including: \n"
        "- 2 to 3 stock investment themes (e.g., defensive stocks, growth sectors)\n"
        "- Rationale behind each theme\n"
        "- Suggested risk level (conservative, balanced, aggressive)\n"
        "- Pros, cons, and hedging considerations"
    )
)

# 3. Ideation Task — Concrete portfolios, no repetition or infinite generation
ideation_task = Task(
    agent=investment_ideation_agent,
    description=(
        "Based on the strategy brief, create exactly three stock portfolio ideas. "
        "Each portfolio should include 3–5 stocks or ETFs aligned to one of the proposed strategies. "
        "Assign allocation weights and provide a short rationale for each holding."
    ),
    expected_output=(
        "A JSON-style output with:\n"
        "- Three named portfolios\n"
        "- Each with 3–5 tickers, allocation weights, and a brief rationale\n"
        "- Clearly mapped to one of the investment strategies"
    )
)

# 4. Content Writing Task — Turn portfolios into readable thought leadership
writing_task = Task(
    agent=writer_agent,
    description=(
        "Create an investment article based on the provided stock portfolios. "
        "Introduce the investment theme, explain the rationale behind selected portfolios, and discuss the risk/reward profile of key stocks. "
        "Use subheadings and bullet points for clarity, and avoid speculative language."
    ),
    expected_output=(
        "A 750–1000 word blog article with:\n"
        "- Engaging introduction and context\n"
        "- Explanation of each strategy and portfolio\n"
        "- Subheadings and bullet points\n"
        "- Neutral, professional tone without speculative advice"
    )
)

# 5. Final Review Task — Avoid indecision by requiring ranking
review_task = Task(
    agent=review_decide_agent,
    description=(
        "Review all outputs including market research, strategy brief, portfolios, and the article. "
        "Rank the three portfolios from most to least suitable for implementation. Select one as the recommended option. "
        "Justify the choice using investment principles (diversification, alignment, risk/reward) and suggest any needed adjustments."
    ),
    expected_output=(
        "A ranked list of all three portfolios with:\n"
        "- Selected top portfolio clearly marked\n"
        "- Justification for the ranking and decision\n"
        "- Optional improvement suggestions (if applicable)"
    )
)


In [7]:
# List all your defined tasks and agents here:
task_list = [research_task, strategy_task, ideation_task, review_task, writing_task]
agent_list = [market_watcher_agent, investment_strategy_agent, investment_ideation_agent, writer_agent, review_decide_agent]



In [8]:

financial_market_advisor_agent = Crew(
    agents=agent_list,
    tasks=task_list,
    process=Process.sequential,
    verbose=True
)

In [9]:
result = financial_market_advisor_agent.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0737d387-6d19-4990-bd32-10fa6b36ba35                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Market Researcher

## Task: Conduct a focused analysis of recent developments in the stock market. Detect and summarize key volatility
triggers such as major earnings surprises, central bank commentary, sector-specific news, geopolitical tensions, or
unusual trading activity. Highlight sectors or stocks potentially impacted.

c:\Users\xrist\Downloads\mesu-ai-main\agenticflow\lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new

...

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.RateLimitError: RateLimitError: OpenAIException - Your account is not active, please check      │
│  your billing details on our website.                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 6417f184-99e3-4658-b941-85a6ff0a8f1a                                                                     │
│  Agent: Market Researcher                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 0737d387-6d19-4990-bd32-10fa6b36ba35                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: RateLimitError: OpenAIException - Your account is not active, please check your billing details on our website.

In [ ]:
print(result)